In [2]:
import time
import re
import requests
from bs4 import BeautifulSoup


In [3]:
def get_html(target_date, page_num):
    """
    주어진 날짜와 페이지 번호에 해당하는 페이지 URL에 접근하여 HTML을 돌려준다.
    :param target_date:
    :param page_num:
    :return:
    """
    user_agent = "'Mozilla/5.0"
    headers ={"User-Agent" : user_agent}

    page_url = "http://news.naver.com/main/list.nhn?sid2=258&sid1=101&mid=shm&mode=LS2D&date=" + \
               str(target_date) + "&page=" + str(page_num) + ""

    response = requests.get(page_url, headers=headers)
    html = response.text

    return html


In [4]:
def ext_news_article_urls(html):
    """
    주어진 html에서 기사 url을 추출하여 돌려준다.
    :param html:
    :return:
    """

    url_frags = re.findall('<a href="(.*?)"', html)
    news_article_urls=[]

    for url_frag in url_frags:
        if  "sid1=101&sid2=258" in url_frag and "aid" in url_frag:
            news_article_urls.append(url_frag)
        else :
            continue

    return news_article_urls


In [5]:
def write_news_article_urls(output_file, urls):
    """
    기사 URL들을 출력 파일에 기록한다.
    :param output_file:
    :param urls:
    :return:
    """
    for url in urls:
        print(url, file=output_file)


In [6]:
def write_html(output_file, html):
    """
    주어진 HTML 텍스트를 출력 파일에 쓴다.
    :param output_file:
    :param html:
    :return:
    """

    output_file.write("{}\n".format(html))
    output_file.write("@@@@@ ARTICLE DELMITER @@@@\n")

In [7]:
target_from_date = "20150101"#시작 날찌 YYYYMMDD
target_to_date = "20191231" #끝나는 날짜 YYYYMMDD
output_file_name = "/content/drive/MyDrive/Colab Notebooks/stock_ML/article_urls" + target_from_date + ".txt"
output_file = open(output_file_name, "w", encoding="utf-8")

In [8]:
i_loop = 0
while True:
    #페이지 몇개 가져올지,,,?
    page_num = 1
    max_page_num = 10

    date = int(target_from_date)+i_loop
    tmp_str_year = str(date)[0:4]
    tmp_str_day = str(date)[6:8]
    tmp_str_month = str(date)[4:6]
    ret = int(tmp_str_month)%2



    if (int(tmp_str_month)%2 == 1 and int(tmp_str_day)>30):
        #홀수 달에서 day 넘기면
        tmp_str_month = int(tmp_str_month)+1
        tmp_str_month = str(tmp_str_month)
        tmp_str_day = '01'
        if(len(tmp_str_month) == 1):
            target_from_date = tmp_str_year+'0'+tmp_str_month+tmp_str_day
            date = int(target_from_date)
        else:
            target_from_date = tmp_str_year +tmp_str_month + tmp_str_day
            date = int(target_from_date)

        i_loop=1

    elif( int(tmp_str_month)%2 == 0 and int(tmp_str_day)>28 and int(tmp_str_month) == 2 ):
        # 2월 달에서 day 넘기면
        tmp_str_month = int(tmp_str_month) + 1
        tmp_str_month = str(tmp_str_month)
        tmp_str_day = '01'
        if(len(tmp_str_month) == 1):
            target_from_date = tmp_str_year+'0'+tmp_str_month+tmp_str_day
            date = int(target_from_date)
        else:
            target_from_date = tmp_str_year +tmp_str_month + tmp_str_day
            date = int(target_from_date)

        i_loop=1

    elif (int(tmp_str_month)%2 == 0 and int(tmp_str_day)>31 and int(tmp_str_month) != 2):
        # 짝수 달에서 day 넘기면
        tmp_str_month = int(tmp_str_month) + 1
        tmp_str_month = str(tmp_str_month)
        tmp_str_day = '01'
        if(len(tmp_str_month) == 1):
            target_from_date = tmp_str_year+'0'+tmp_str_month+tmp_str_day
            date = int(target_from_date)
        else:
            target_from_date = tmp_str_year +tmp_str_month + tmp_str_day
            date = int(target_from_date)

        i_loop=1

    if (int(tmp_str_month) > 12):
        # if it is over december
        tmp_str_year = int(tmp_str_year) + 1
        tmp_str_year = str(tmp_str_year)
        tmp_str_month = '01'
        tmp_str_day = '01'
        target_from_date = tmp_str_year + tmp_str_month + tmp_str_day
        date = int(target_from_date)

    else:
        i_loop+=1

    #end point
    if(str(target_from_date[0:4])=='2020'):
        break


    while True :
        html = get_html(date, page_num)

        if page_num>=max_page_num:
            break

        urls = ext_news_article_urls(html)
        #셋으로 중복 된 거 빼내고
        urls = set(urls)

        write_news_article_urls(output_file, urls)
        page_num+=1
        # pause()

output_file.close(output_file)


KeyboardInterrupt: ignored

In [9]:
url_file_name = "/content/drive/MyDrive/Colab Notebooks/stock_ML/article_urls20150101.txt"#input("Enter url file name : ")
output_file_name =  "/content/drive/MyDrive/Colab Notebooks/stock_ML/html20150101.txt"#input("Enter output file name : ")
url_file = open(url_file_name, "r", encoding ="utf-8")
output_file = open(output_file_name, "w", encoding='utf-8')
target_word = "삼성전자" #input("Enter keyword : ")


In [10]:

for line in url_file:

    article_id = line[(len(line) - 24):len(line)]
    print_url = "http://news.naver.com/main/tool/print.nhn?" + article_id


    user_agent = "'Mozilla/5.0"
    headers = {"User-Agent": user_agent}

    response = requests.get(print_url, headers=headers)
    html = response.text

    html_2 = BeautifulSoup(html, "html.parser")
    articles = html_2.select("div.article_header > h3")
    title = articles[0].contents[0]
    if (target_word in title):
        print("있음")
        write_html(output_file, html)


output_file.close(output_file)
url_file.close(url_file)



있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음
있음


KeyboardInterrupt: ignored

In [12]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
네이버 뉴스 기사 HTML에서 순수 텍스트 기사를 추출한다.
"""

import bs4
import time
import re
import requests
import os


ARTICLE_DELIMITER = "@@@@@ ARTICLE DELMITER @@@@\n"
TITLE_START_PAT = '<h3 class="font1">'
TITLE_END_PAT = '</h3>'
DATE_TIME_START_PAT = u'기사입력 <span class="t11">'
BODY_START_PAT = '<div class="article_body">'
BODY_END_PAT = '<div class="article_footer">'
TIDYUP_START_PAT = '<div class="article_footer">'


def read_html_article(html_file):
    """
    HTML 파일에서 기사 하나를 읽어서 돌려준다.
    :param html_file:
    :return:
    """

    lines = []
    for line in html_file:
        if line.startswith(ARTICLE_DELIMITER):
            html_text = "".join(lines).strip()
            return html_text
        lines.append(line)

    return None

def ext_title(html_text):
    """
    HTML 기사에서 제목을 추출하여 돌려준다.
    :param html_text:
    :return:
    """
    p = html_text.find(TITLE_START_PAT)
    q = html_text.find(TITLE_END_PAT)
    title = html_text[p + len(TITLE_START_PAT):q]
    title = title.strip()

    return title


def ext_date_time(html_text):
    """
    HTML 기사에서 날짜와 시간을 추출하여 돌려준다.
    :param html_text:
    :return:
    """
    start_p = html_text.find(DATE_TIME_START_PAT)+len(DATE_TIME_START_PAT)
    end_p = start_p + 10
    date_time = html_text[start_p:end_p]
    date_time = date_time.strip()

    return date_time

def strip_html(html_body):
    """
    HTML 본문에서 HTML 태그를 제거하고 돌려준다.
    :param html_body:
    :return:
    """
    page = bs4.BeautifulSoup(html_body, "html.parser")
    body = page.text

    return body

def tidyup(body):
    """
    본문에서 필요없는 부분을 자르고 돌려준다.
    :param body:
    :return:
    """

    p = body.find(TIDYUP_START_PAT)
    body = body[:p]
    body = body.strip()

    return body

def ext_body(html_text):
    """
    HTML 기사에서 본문을 추출하여 돌려준다.
    :param html_text:
    :return:
    """

    p = html_text.find(BODY_START_PAT)
    q = html_text.find(BODY_END_PAT)
    html_body = html_text[p + len(BODY_START_PAT):q]
    len(html_body)
    href_inx = html_body.find('<a')
    # ^ M_$
    html_body = html_body[0:href_inx-1]
    html_body = html_body.replace("<br />","\n")
    html_body = html_body.strip()
    body = strip_html(html_body)
    body = tidyup(body)

    return body

def write_article(text_file, title, date_time, body):
    """
    텍스트 파일에 항목이 구분된 기사를 출력한다.
    :param text_file:
    :param title:
    :param date_time:
    :param body:
    :return:
    """

    text_file.write("{}\n".format(title))
    # text_file.write("{}\n".format(date_time))
    text_file.write("{}\n".format(body))
    # text_file.write("{}\n".format(ARTICLE_DELIMITER))

In [13]:
html_file_name = "/content/drive/MyDrive/Colab Notebooks/stock_ML/html20150101.txt"#input("Enter HTML File name : ")
text_file_name ="/content/drive/MyDrive/Colab Notebooks/stock_ML/result20150101.txt" #input("Enter text file name : ")

html_file = open(html_file_name, "r", encoding="utf-8")
text_file = open(text_file_name, "w", encoding="utf-8")

while True:

    html_text = read_html_article(html_file)

    if not html_text:
        break

    title = ext_title(html_text)
    date_time = ext_date_time(html_text)
    body = ext_body(html_text)
    write_article(text_file, title, date_time, body)

html_file.close()
text_file.close()
